In [16]:
import csv
import time
from polygon import RESTClient
from polygon.exceptions import BadResponse
from datetime import date, timedelta, datetime
import pandas as pd

client = RESTClient("flAcFuvYhLgjSRPblks2_lXXvTi5ALjk") # Replace with your actual API key

ticker = 'GOOG'
start_date_str = '2024-03-01'
end_date_str = '2025-05-30'

output_csv_filename = f"{ticker}_daily_data_{start_date_str}_to_{end_date_str}.csv"

all_data_for_df = []

print(f"Fetching daily aggregates for {ticker} from {start_date_str} to {end_date_str}...")

try:
    for agg in client.list_aggs(
        ticker=ticker,
        multiplier=1,
        timespan='day',
        from_=start_date_str, # Changed 'from' to 'from_'
        to=end_date_str,
        adjusted=True
    ):
        # Explicitly ensure timestamp is a datetime object
        # The timestamp from Polygon is in milliseconds, so divide by 1000
        if isinstance(agg.timestamp, int):
            agg_datetime = datetime.fromtimestamp(agg.timestamp / 1000)
        elif isinstance(agg.timestamp, datetime):
            agg_datetime = agg.timestamp
        else:
            print(f"Warning: Unexpected type for agg.timestamp: {type(agg.timestamp)}. Skipping this aggregate.")
            continue

        row_dict = {
            'Date': agg_datetime.date().isoformat(),
            'Open': agg.open,
            'High': agg.high,
            'Low': agg.low,
            'Close': agg.close,
            'Volume': agg.volume,
            'VWAP': agg.vwap,
            'Transactions': agg.transactions,
            'Ticker': ticker
        }
        all_data_for_df.append(row_dict)
        print(agg) # This will print each aggregate object as it's fetched

except BadResponse as e:
    print(f"Error fetching data: {e.message}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# --- Save to CSV using Pandas ---
if all_data_for_df:
    column_order = [
         'Date', 'Ticker', 'Open', 'High', 'Low', 'Close',
        'Volume', 'VWAP', 'Transactions'
    ]

    df = pd.DataFrame(all_data_for_df, columns=column_order)

    try:
        df.to_csv(output_csv_filename, index=False)
        print(f"\nSuccessfully saved data to {output_csv_filename}")
    except IOError as e:
        print(f"Error writing CSV file: {e}")
else:
    print("\nNo data collected to save to CSV.")


Fetching daily aggregates for GOOG from 2024-03-01 to 2025-05-30...
Agg(open=139.61, high=140, low=137.975, close=138.08, volume=28551525.0, vwap=138.7688, timestamp=1709269200000, transactions=314669, otc=None)
Agg(open=136.54, high=136.63, low=132.86, close=134.2, volume=43571510.0, vwap=134.0105, timestamp=1709528400000, transactions=495654, otc=None)
Agg(open=132.74, high=134.02, low=131.55, close=133.78, volume=28447550.0, vwap=133.0401, timestamp=1709614800000, transactions=343013, otc=None)
Agg(open=134.24, high=134.74, low=131.95, close=132.56, volume=23175200.0, vwap=132.7237, timestamp=1709701200000, transactions=302445, otc=None)
Agg(open=133.89, high=135.82, low=132.66, close=135.24, volume=24107282.0, vwap=134.7866, timestamp=1709787600000, transactions=279499, otc=None)
Agg(open=135.035, high=138.985, low=134.8, close=136.29, volume=26495160.0, vwap=137.1159, timestamp=1709874000000, transactions=333673, otc=None)
Agg(open=137.07, high=139.98, low=137.07, close=138.94, vo